In [225]:
#import libraries used into
from datetime import datetime, timedelta
import pandas as pd, numpy as np, os

In [226]:
DS = {
    'Test' : ['1','2','3'],
    'Results' : ['1','2','3']
}
DF = pd.DataFrame(DS)
print(type(DF))

<class 'pandas.core.frame.DataFrame'>


In [227]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
print("fileam: ", fileam)
fileyest = "OT " + (datetime.today() - timedelta(days = 4)).strftime('%d.%m.%Y' + ".XLSX")
print(fileyest)
"""
if datetime.today().strftime('%A') == "Monday":
    fileyest = "OT " + (datetime.today() - timedelta(days = 3)).strftime('%d.%m.%Y' + ".XLSX")
    print("fileyest: ", fileyest)
else:
    fileyest = "OT " + (datetime.today() - timedelta(days = 1)).strftime('%d.%m.%Y' + ".XLSX")
    print("fileyest: ", fileyest)
"""
main_path = "C:\\Users\\mruiz\\Documents\\DB OT\\" 
current_path = "C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)
data_yesterday_excel = {}
data_am_excel = {}
repro_nuevas = 1

fileam:  OT 22.11.2022.XLSX
OT 18.11.2022.XLSX


In [228]:
def get_data(entry_data):
    file = main_path + entry_data
    filexcel = pd.read_excel(file,sheet_name="Sheet1")
    print("Los datos del archivo ", entry_data, " fueron procesados")
    return filexcel

In [229]:
while True:
    try:
        data_yesterday_excel = get_data(fileyest)
        data_am_excel = get_data(fileam)
        break
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        break

Los datos del archivo  OT 18.11.2022.XLSX  fueron procesados
Los datos del archivo  OT 22.11.2022.XLSX  fueron procesados


In [230]:
if type(data_am_excel) == type(data_yesterday_excel):
    programa = data_am_excel
    print("Los datos de la mañana fueron seleccionados")
else :
    print("Error")

Los datos de la mañana fueron seleccionados


In [231]:
data2 =current_path + "\\Master_OT.xlsx"
nombres = pd.read_excel(data2, sheet_name="Names", header=0)

In [232]:
data3 = current_path + "\\Politicas.XLSX"
politicas = pd.read_excel(data3, sheet_name="POLITICAS CONTRATO", header=0)

In [233]:
data7 = current_path + "\\Ubicaciones_Equipos.xlsx"
ubicaciones = pd.read_excel(data7, sheet_name="Sheet1", header=0)

In [234]:
data8 = current_path + "\\Reprogramaciones.xlsx"
reprogramaciones_ = pd.read_excel(data8, sheet_name="DB", header=0)

In [235]:
reprogramaciones = data_yesterday_excel.copy()

In [236]:
reprogramaciones.rename(columns={'Inicio programado':'Fecha_reprogramada'}, inplace=True)

In [237]:
programa['Material'] = np.where((programa['Clase de orden'] == 'ZMCP') &  (programa['Texto breve'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(programa['Clase de orden'] == 'ZPM2','SI',
np.where(programa['Clase de orden'] == 'ZMCP','SI',
np.where(programa['Clase de orden'] == 'ZMCI', 'SI',
np.where((programa['Clase de orden'] == 'ZPM1') &  (programa['Texto breve'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [238]:
#print('Las columnas de programa son', list(programa.columns.values))
#print('Las columnas de nombres son', list(nombres.columns.values))
#print('Las columnas de politicas son', list(politicas.columns.values))
#print('Las columnas de material local son', list(mlocal.columns.values))
#print('Las columnas de material foraneo son', list(mforaneo.columns.values))
#print('Las columnas de las repro son ',list(reprogramaciones.columns.values))
#print('Las columnas de reportes son ', list(reportes.columns.values))
#print('Las columnas de ubicaciones son', list(ubicaciones.columns.values))
#print(list(result_df.columns.values))

COMIENZAN LOS MERGE

In [239]:
repro_columns = ['Fecha_reprogramada', 'Orden']

In [240]:
result_df = pd.merge(programa,reprogramaciones[repro_columns],on='Orden',how='left' )

In [241]:
result_df = pd.merge(left=result_df,right=nombres,left_on='Pto.tbjo.responsable', right_on='TecID',how='left')

In [242]:
politicas_columns = ['Equipo','gestora archivo anahi ', 'reportes archivo anahi ', 'Tipo Equipo', 'Frecuencia', 
'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3', 
'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba', 'Tecnicos para servicio', 'Requerimientos adicionales', 
'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC', 'FRECUENCIA DE OC', 
'PREVENTIVOS', 'CORRECTIVOS']

In [243]:
result_df = pd.merge(result_df,politicas[politicas_columns],on='Equipo',how='left')

In [244]:
ubicaciones_columns = ['Equipo', 'Dirección', 'Latitud', 'Longitud']

In [245]:
result_df = pd.merge(result_df,ubicaciones[ubicaciones_columns],on='Equipo',how='left')

TERMINAN LOS MERGE

In [246]:
print(list(result_df.columns.values))

['Inicio programado', 'Orden', 'Plan mant.preventivo', 'Pto.tbjo.responsable', 'Status del sistema', 'Texto breve', 'Cliente', 'Denominación de objeto técnico', 'Número de serie', 'Equipo', 'Clase de orden', 'Fe.liberación real', 'Fecha inicio real', 'Material', 'Fecha_reprogramada', 'TecID', 'Short name', 'Estatus', 'City', 'Center', 'gestora archivo anahi ', 'reportes archivo anahi ', 'Tipo Equipo', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba', 'Tecnicos para servicio', 'Requerimientos adicionales', 'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS', 'Dirección', 'Latitud', 'Longitud']


In [247]:
result_df.assign(reprogramacion=0);

In [248]:
reprogramaciones = result_df.copy()

In [249]:
result_df['reprogramacion'] = np.where(result_df['Inicio programado']==reprogramaciones['Fecha_reprogramada'], "Sin_Reprogramar","Reprogramada")
result_df['reprogramacion'] = np.where(result_df['Fecha_reprogramada'].isnull(),"Sin estatus",result_df['reprogramacion'])

In [260]:
print(result_df['reprogramacion'].unique())

['Sin_Reprogramar' 'Reprogramada' 'Sin estatus']


In [261]:
result_df['Politica'] = np.where(result_df['Requiere Acceso'].isnull(),"NO","SI")
result_df['Servicio'] = np.where(result_df['Orden'].isnull(), int("0"),int("1"))

In [262]:
print(result_df.shape[0])
print(programa.shape[0])

40967
40967


In [263]:
reprogramaciones = result_df[result_df['reprogramacion'].isin(['Reprogramada'])]
if datetime.today().strftime('%Y-%m-%d') == reprogramaciones_['Dia_reprogramado'].loc[0]:
       repro_nuevas = 0
       print("Sin Reprogramaciones nuevas")
       reprogramaciones = reprogramaciones_
else:
       reprogramaciones = reprogramaciones.assign(Dia_reprogramado = datetime.today().strftime('%Y-%m-%d'))
       reprogramaciones = pd.concat([reprogramaciones,reprogramaciones_])
       print(reprogramaciones.shape[0])

Sin Reprogramaciones nuevas


In [264]:
reprogramaciones = reprogramaciones[['Orden', 'Inicio programado','Plan mant.preventivo', 'Pto.tbjo.responsable',
       'Status del sistema', 'Texto breve', 'Cliente', 'Fecha inicio real',
       'Fecha_reprogramada','Dia_reprogramado']]

In [265]:
if repro_nuevas == 0 :
    print("Se conservó el documento")
elif repro_nuevas != 0 : 
    os.remove(current_path + "\\" + "Reprogramaciones.xlsx")
    path = current_path + "\\" + "Reprogramaciones.xlsx"   
    reprogramaciones.to_excel(path, index=False,header=True)
    with pd.ExcelWriter(path) as writer:
        reprogramaciones.to_excel(writer,index=False,sheet_name="DB")
    os.startfile(path)

Se conservó el documento


In [266]:
print(list(result_df.columns.values))

['Inicio programado', 'Orden', 'reprogramacion', 'Fecha_reprogramada', 'Plan mant.preventivo', 'Pto.tbjo.responsable', 'Short name', 'Estatus', 'City', 'Status del sistema', 'Servicio', 'Fe.liberación real', 'Fecha inicio real', 'Texto breve', 'Cliente', 'Denominación de objeto técnico', 'Número de serie', 'Equipo', 'Dirección', 'Politica', 'gestora archivo anahi ', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba', 'Tecnicos para servicio', 'Requerimientos adicionales', 'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS', 'Material', 'Clase de orden']


In [267]:
result_df = result_df[['Inicio programado', 'Orden', 'reprogramacion', 'Fecha_reprogramada',    'Plan mant.preventivo', 'Pto.tbjo.responsable',
 'Short name','Estatus', 'City', 'Status del sistema','Servicio','Fe.liberación real', 'Fecha inicio real', 'Texto breve',
'Cliente', 'Denominación de objeto técnico', 'Número de serie', 'Equipo', 'Dirección',
'Politica', 'gestora archivo anahi ', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba','Tecnicos para servicio', 'Requerimientos adicionales',
'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC','FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS',
'Material', 'Clase de orden']]

In [268]:
os.remove(current_path + "\\" + "Main.xlsx")
path = current_path + "\\" + "Main.xlsx"   
result_df.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")
os.startfile(path)